In [91]:
import torch
from torch import nn
from torch.optim import Adam
import numpy as np
import torch.nn.functional as F
import pandas as pd
#from optimization import Adam

import torch
import torch_scatter
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric as pyg
import torch_geometric.nn as pyg_nn
import torch_geometric.utils as pyg_utils

from torch import Tensor
from typing import Union, Tuple, Optional
from torch_geometric.typing import (OptPairTensor, Adj, Size, NoneType,
                                    OptTensor)

from torch.nn import Parameter, Linear
from torch_sparse import SparseTensor, set_diag
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.utils import remove_self_loops, add_self_loops, softmax













###########################################################################





class MAPredNet(nn.Module):
    def __init__(self):
        super(MAPredNet, self).__init__()
        # hyperparams
        self.s_year = 1997
        self.e_year = 2020
        self.a_freq_fv_dim = 14
        self.target_fv_dim = 17
        self.embedding_b = 32
        self.embedding_c = 16
        self.embedding_z = 32
        self.dropout_ratio = 0.25



        # define models
        self.timing_net = Timing_Net(self.embedding_b, self.embedding_c)
        # # input: (L1, a_freq_fv_dim); output: (L1, embedding_b)
        self.b_net = nn.Sequential(
                        nn.Linear(in_features=self.a_freq_fv_dim, out_features=64), 
                        nn.Dropout(self.dropout_ratio), 
                        nn.ReLU(),
                        nn.Linear(in_features=64, out_features=self.embedding_b))         
        self.c_net =  nn.GRU(input_size = 3, hidden_size = self.embedding_c, batch_first=True)
        self.choice_net = ChoiceNet(self.embedding_b, self.embedding_c, self.embedding_z)




    def MCestimator(self, arr, estimate_length):
        '''
        Monte Carlo Estimator 
        input: 
            arr: 1d arr
            estimate_length: scalar
        '''
        estimation =  estimate_length*(1/(arr.size(0)-1))*torch.sum(arr)
        
        return estimation

    def likelihood(self, event_time_ll, non_event_time_ll):
        '''
        compute likelihood loss
        input are all scalars
        loss: small = good
        '''
        loss = - event_time_ll + non_event_time_ll
        return loss

    def forward(self, arr_b, arr_c, arr_delta_time, event_data, non_event_data, estimate_length, choice_data_dict):
        '''
        WARNING: 
            1. This Version only works for batch_size = 1 (for a single firm)..... (if batch size > 1, have to padding at first to make arr_b, arr_c has same size
                    if padding, the number of event of every year for all firms should be the same (in dict_idx[year]))
            2. all idx must be put into list in time order
            3. year is always an integer

        Assume:
        L1 = Length_of_year_cross
        L2 = length_of_self+peer_events
        L3 = length_of_self event
        L_Neg = Length of negative samples

        N_i_2 = N_self_event_ith_year


        Inputs:
            arr_b: raw financial variables; (L1, 14) 

            arr_c: raw peer/self effect variables; (L2, 3)

            arr_delta_time: array of delta_time; (L3, 1); 
                arr_delta_time is corresponding to arr_c: time delta in i row = (t_event_i+1 - t_event_i) 

            event_data: 
                arr_b_idx: lst of indeces, [3, 3, 4, 4, 4, 5, 9 ...]
                    length = L3
                    element: integer as row number in arr_b; for true event
                arr_c_idx: lst of indeces, [3, 3, 4, 4, 4, 5, 9 ...]
                    length = L3
                    element: integer as row number in arr_c; for true event
                arr_delta_time:  (same as...)
                    array !! not idx  (L3, )
                    

                    

                    

            
            non_event_data: for negative sampling in timing model only; idea: draw time point from Unif(0, MAX_T). 
                                pick up the corresponding b, c, delta_t.
                arr_b_idx:  lst of indeces, 
                    length: L_Neg
                arr_c_idx: 
                arr_t_non: array, not index!
                

            estimate_length: scalar, for negative sampling MC estimator, 
                max(time)  - min(time)


            choice_data_dict: for choice model only, a dict contains:(invariant for firms,  variant by year)
                - dict_idx: A dict split arr_b_idx and arr_c_idx to year. e.g. {1997: [true_tar_idxs, arr_b_idx, arr_c_idx], '1998': [arr_b_idx, arr_c_idx], ...} 
                            year is "the year that self event happens" (so the size varies for differnt firms)
                    

                    - arr_b_idx_i: lst, N_i_2 
                    - arr_c_idx_i: lst, N_i_2 
                
                - true_tar_idxs_i: torch tensor, one-hot, size = (N_i_2, N_i_1) # for each row, only 1 (true acquirer) is 1, others are 0.
                        N_i_1 = N_i (every firm could be in target candidate)
                - node features np.array: [N_i, in_channels_i]  # inchannels = 
                - network structure np.array(edges): [2, N_edges_i] # the idx here is corresponding to node feature array

                
        output:
        
        '''
        # check input data 
        # arr_b_idx, arr_c_idx, arr_delta_time = event_data # expand
        # arr_b_idx_non, arr_c_idx_non, arr_delta_time_non = non_event_data
        # assert (len(arr_b_idx) == len(arr_c_idx)), "the size of input indeces dismatch for event data"
        # assert (len(arr_b_idx_non) == len(arr_c_idx_non)), "the size of input indeces dismatch for non-event data"
        # assert (arr_c.size[0] == arr_delta_time.size[0]), "the size of input array dismatch"


        # transform to embedding
        #arr_b = arr_b
        #print(type(arr_b.squeeze().numpy()[0, 0]))
        mat_b = self.b_net(arr_b) # (L1, embedding_b)
        mat_c = self.c_net(arr_c) # (L2, embedding_c)
        

        # timing model
        event_lambda = self.timing_net(mat_b, mat_c, event_data) # (L3, )
        non_event_lambdas = self.timing_net(mat_b, mat_c,  non_event_data) # (L_Neg, )

        event_time_ll = torch.sum(torch.log(event_lambda))  # out = scalar
        non_event_time_ll = self.MCestimator(non_event_lambdas, estimate_length)  # out = scalar

        log_likelihood_loss = self.likelihood(event_time_ll, non_event_time_ll)  # out = scalar
        

        # choice model
        event_choice_loss = self.choice_net(mat_b, mat_c, choice_data_dict, self.s_year, self.e_year)

        total_loss = log_likelihood_loss + event_choice_loss
        return total_loss, log_likelihood_loss, event_choice_loss
    


class Timing_Net(nn.Module):
    def __init__(self, embedding_b, embedding_c):
        super(Timing_Net, self).__init__()
        self.phi =  torch.randn(1, requires_grad=True)
        self.w_b =  torch.randn(embedding_b, requires_grad=True) # (embedding_b, )
        self.w_c =  torch.randn(embedding_b, requires_grad=True)# (embedding_c, )
        self.omega = torch.randn(1, requires_grad=True)# scalar

    def f_lambda(self, x):
        '''
        inputs
        '''
        lambda_dt = self.phi*torch.log(1+torch.exp(x/self.phi))

        return lambda_dt

    def forward(self, mat_b, mat_c, arr_delta_time, event_data):
        '''
        Assume:
            L1 = Length_of_year_cross
            L2: length_of_self+peer_events
            L3: length_of_self event
            
        Input: 
            mat_b: (L1, embedding_b)
            mat_c: (L2, embedding_c)
            arr_delta_time: (L2, embedding_c)

        
        '''
        arr_b_idx, arr_c_idx, arr_delta_t = event_data # expand
        b = mat_b[arr_b_idx] # (L3, embedding_b)
        c = mat_c[arr_c_idx] # (L3, embedding_c)
        delta_t = arr_delta_t # (L3, 1)

        # (L3, )
        rate = torch.matmul(b, self.w_b)  + torch.matmul(c, self.w_c) + self.omega * torch.exp(-self.omega * delta_t) 
        lambda_dt = self.f_lambda(rate)  # (L3, )
        return lambda_dt









class ChoiceNet(nn.Module):
    
    def __init__(self, embedding_b, embedding_c, embedding_z):
        super(ChoiceNet, self).__init__()
        # hyperparams
        self.embedding_z = embedding_z
        self.embedding_b, self.embedding_c = embedding_b, embedding_c
        self.gnn_hidden_dim = 64
        self.gnn_model_type = "GraphSage" # GraphSage or GAT
        self.target_fv_dim = 17
        self.dropout_ratio = 0.25

        # build modules
        self.gnn_choice = GNN_Stack(self.target_fv_dim, self.gnn_hidden_dim, self.embedding_z, self.gnn_model_type)
        self.transform = nn.Sequential(  # input dim = (N_of_self_event, embedding_b + embedding_c), output dim = (N_of_self_event, embedding_z)
                        nn.Linear(in_features=self.embedding_b + self.embedding_c, out_features=64), nn.Dropout(self.dropout_ratio), nn.ReLU(),
                        nn.Linear(in_features=64, out_features=self.embedding_z))
        self.loss = nn.BCELoss()




    def forward(self, mat_b, mat_c, choice_data_dict, s_year, e_year):
        '''
        input:
            choice_data_dict:  {1997: }
                length of dict: number of years (for all firms, always pass the full dataset into ChoiceNet)
         
        '''
        choice_loss_lst = []
        for year in range(s_year, e_year):
            '''
            At the ith iteration of the loop,
                compute binary cross entropy loss for choice problem of i-th year
                based on all of the MA event occurred in that year
            
            N_i_1 = number of candidate target in i-th year
            N_i_2 = number of self events in i-th year

            '''
            dict_idx, true_tar_idxs_i, features_i, edges_i = choice_data_dict[year] # list=:[N_i_1], arrays: (N_i_1, 22) , (2, |E|)
            arr_b_idx_i, arr_c_idx_i = dict_idx 
            # true_tar_idxs_i: tensor, one-hot, size = (N_i_2, N_i_1)
            # arr_b_idx_i, arr_c_idx_i: list: length = N_i_2 and N_i_2 
            # if there's no self event in ith year, continue
            if len(arr_b_idx_i == 0): 
                continue
            else:
                # GNN part
                '''
                always pass the entire graph for i-th year into GNN
                '''
                
                #assert len(true_tar_idxs_i) == features_i.size(0), "number of self events mismatch in choice data"
                z_vt_i = self.gnn_choice(features_i, edges_i) # (N_i_1, embedding_z)

                # z_dt : (N_i_2, embedding_z)
                z_dt = self.transform(torch.cat((mat_b[arr_b_idx_i], mat_c[arr_c_idx_i]), dim=1))  # z_dt : (N_i_2, embedding_b + embedding_c)

                # broadcasting
                z_vt_i = z_vt_i.unsqueeze(0) # (1, N_i_1, embedding_z)
                z_dt_i = z_dt_i.unsqueeze(1) # (N_i_2, 1, embedding_z)
                logits_i = (z_dt_i * z_vt_i).sum(axis=-1) # (N_i_2, N_i_1)
                choice_l = self.loss(nn.Sigmoid(logits_i), true_tar_idxs_i)  # inputs are both (N_i_2, N_i_1)
                choice_loss_lst.append(choice_l) # append a scalar

        return torch.sum(choice_loss_lst)


            
            





















                
##################### GNN ###########################################
            


class GNN_Stack(torch.nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, model_type, emb=True):
        super(GNN_Stack, self).__init__()

        # arguments
        self.model_type = model_type
        self.num_layers = 2
        self.heads = 1
        self.dropout = 0.25
        





        conv_model = self.build_conv_model(self.model_type)
        self.convs = nn.ModuleList()
        self.convs.append(conv_model(input_dim, hidden_dim))
        assert (self.num_layers >= 1), 'Number of layers is not >=1'
        for l in range(self.num_layers-1):
            self.convs.append(conv_model(self.heads * hidden_dim, hidden_dim))

        # post-message-passing
        self.post_mp = nn.Sequential(
            nn.Linear(self.heads * hidden_dim, hidden_dim), nn.Dropout(self.dropout), 
            nn.Linear(hidden_dim, output_dim))

        self.dropout = self.dropout
        self.num_layers = self.num_layers

        self.emb = emb

    def build_conv_model(self, model_type):
        if model_type == 'GraphSage':
            return GraphSage 
        elif model_type == 'GAT':
            return GAT

    def forward(self, X, E):
        '''
        X: the node features: [N, input_dim]
        E: the network structure: [2, E]  # note that the idx is corresponding to X


        tar_net_fv_i, tar_net_E_i
        
        '''
        x, edge_index = X, E 
          
        for i in range(self.num_layers):
            x = self.convs[i](x, edge_index)
            x = F.relu(x)
            x = F.dropout(x, p=self.dropout)

        x = self.post_mp(x)

        if self.emb == True:
            return x

        return F.log_softmax(x, dim=1)


class GraphSage(MessagePassing):
    
    def __init__(self, in_channels, out_channels, normalize = True,
                 bias = False, **kwargs):  
        super(GraphSage, self).__init__(**kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.normalize = normalize

        self.lin_l = nn.Linear(self.in_channels, self.out_channels)
        self.lin_r = nn.Linear(self.in_channels, self.out_channels)

        

        self.reset_parameters()


    def reset_parameters(self):
        self.lin_l.reset_parameters()
        self.lin_r.reset_parameters()

    def forward(self, x, edge_index, size = None):
        prop = self.propagate(edge_index, x=(x, x), size=size)
        out = self.lin_l(x) + self.lin_r(prop)
        if self.normalize:
            out = F.normalize(out, p=2)

        return out

    def message(self, x_j):
        out = x_j
        return out

    def aggregate(self, inputs, index, dim_size = None):
        # The axis along which to index number of nodes.
        node_dim = self.node_dim
        out = torch_scatter.scatter(inputs, index, node_dim, dim_size=dim_size, reduce='mean')

        return out







class GAT(MessagePassing):

    def __init__(self, in_channels, out_channels, heads = 2,
                 negative_slope = 0.2, dropout = 0., **kwargs):
        super(GAT, self).__init__(node_dim=0, **kwargs)

        self.in_channels = in_channels
        self.out_channels = out_channels
        self.heads = heads
        self.negative_slope = negative_slope
        self.dropout = dropout

        self.lin_l = None
        self.lin_r = None
        self.att_l = None
        self.att_r = None


        # self.lin_l is the linear transformation that you apply to embeddings 
        self.lin_l = nn.Linear(self.in_channels, self.out_channels * self.heads)
        self.lin_r = self.lin_l

        self.att_l = nn.Parameter(torch.zeros(self.heads, self.out_channels))
        self.att_r = nn.Parameter(torch.zeros(self.heads, self.out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        nn.init.xavier_uniform_(self.lin_l.weight)
        nn.init.xavier_uniform_(self.lin_r.weight)
        nn.init.xavier_uniform_(self.att_l)
        nn.init.xavier_uniform_(self.att_r)

    def forward(self, x, edge_index, size = None):
        
        H, C = self.heads, self.out_channels


        x_l = self.lin_l(x).reshape(-1, H, C)
        x_r = self.lin_r(x).reshape(-1, H, C)
        alpha_l = self.att_l * x_l
        alpha_r = self.att_r * x_r
        out = self.propagate(edge_index, x=(x_l, x_r), alpha=(alpha_l, alpha_r), size=size)
        out = out.reshape(-1, H*C)

        return out




    def message(self, x_j, alpha_j, alpha_i, index, ptr, size_i):

        alpha = F.leaky_relu(alpha_i + alpha_j, negative_slope=self.negative_slope)
        if ptr:
            att_weight = F.softmax(alpha_i + alpha_j, ptr)
        else:
            att_weight = pyg.utils.softmax(alpha_i + alpha_j, index)
        att_weight = F.dropout(att_weight, p=self.dropout)
        out = att_weight * x_j


        return out


    def aggregate(self, inputs, index, dim_size = None):
        out = torch_scatter.scatter(inputs, index, self.node_dim, dim_size=dim_size, reduce='sum')
    
        return out



    


        
        

In [92]:
model = MAPredNet()
for name, param in model.named_parameters():
    if param.requires_grad:
        print(name, param.dtype)


b_net.0.weight torch.float32
b_net.0.bias torch.float32
b_net.3.weight torch.float32
b_net.3.bias torch.float32
c_net.weight_ih_l0 torch.float32
c_net.weight_hh_l0 torch.float32
c_net.bias_ih_l0 torch.float32
c_net.bias_hh_l0 torch.float32
choice_net.gnn_choice.convs.0.lin_l.weight torch.float32
choice_net.gnn_choice.convs.0.lin_l.bias torch.float32
choice_net.gnn_choice.convs.0.lin_r.weight torch.float32
choice_net.gnn_choice.convs.0.lin_r.bias torch.float32
choice_net.gnn_choice.convs.1.lin_l.weight torch.float32
choice_net.gnn_choice.convs.1.lin_l.bias torch.float32
choice_net.gnn_choice.convs.1.lin_r.weight torch.float32
choice_net.gnn_choice.convs.1.lin_r.bias torch.float32
choice_net.gnn_choice.post_mp.0.weight torch.float32
choice_net.gnn_choice.post_mp.0.bias torch.float32
choice_net.gnn_choice.post_mp.2.weight torch.float32
choice_net.gnn_choice.post_mp.2.bias torch.float32
choice_net.transform.0.weight torch.float32
choice_net.transform.0.bias torch.float32
choice_net.transfo

In [93]:
#dataset = MADataset()
#loader = DataLoader(dataset, batch_size=1, shuffle=True)

In [94]:
epochs = 50
data_size = len(dataset)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

# build model
model = MAPredNet()

model = model.float()
#model = model.to(device)
opt = torch.optim.Adam(params=model.parameters(), lr=0.01)

for epoch in range(epochs):
    loss_e = 0
    timing_loss_e = 0
    choice_loss_e = 0

    model.train()
    for batch in loader:
        batch = tuple(batch)
        arr_b, arr_c, arr_delta_time, event_data, non_event_data, estimate_length, choice_data_dict = batch
        

        opt.zero_grad()
        loss, timing_loss, choice_loss  = model(arr_b, arr_c, arr_delta_time, event_data, non_event_data, estimate_length, choice_data_dict)
        loss.backward()
        opt.step()
        loss_e += loss
        timing_loss_e += timing_loss
        choice_loss_e += choice_loss
        break
    
    loss_e /= data_size
    timing_loss_e /= data_size
    choice_loss_e /= data_size
    # writer.add_scalar('training timing loss', timing_loss_e, epoch)
    # writer.add_scalar('training choice loss', choice_loss_e, epoch)
    # writer.add_scalar('total loss', loss_e, epoch)

    print("Epoch {}. Total Loss: {:.4f}. Timing MLE loss: {:.4f}. Choice BCE loss {:.4f}".format(
            epoch, loss_e, timing_loss_e, choice_loss_e))

    

<class 'numpy.float64'>


RuntimeError: expected scalar type Double but found Float

In [83]:
tt = np.single(np.array([[1],[2]]))


In [84]:
type(tt[0,0])

numpy.float32

In [81]:
tt1 = np.array([[1.],[2.]])

In [82]:
type(tt1[0,0])

numpy.float64